In [ ]:
# dataset used is available on kaggle : https://www.kaggle.com/competitions/e-commerce-shoppers-behaviour-understanding/data
# Input data files are available in the read-only "../input/" directory
import numpy as np 
import pandas as pd 
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

Copy train data set in "train_data" and test dataset in "test_data"

In [ ]:
test_data=pd.read_csv("/kaggle/input/e-commerce-shoppers-behaviour-understanding/test_data_v2.csv")
train_data=pd.read_csv("/kaggle/input/e-commerce-shoppers-behaviour-understanding/train_data_v2.csv")

In [ ]:
train_data.info()

seperate label column "Made_Purchase" from the train dataset

In [ ]:
train_features = train_data.drop("Made_Purchase", axis=1)
train_label = train_data["Made_Purchase"].copy()

change categorical values to indicator variables

In [ ]:
train_dummy = pd.get_dummies(train_features)
test_dummy = pd.get_dummies(test_data)

In [ ]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy="mean")

fill in values in null data points

In [ ]:
imputer.fit(train_dummy)
train_features2 = imputer.transform(train_dummy)
train_features_df = pd.DataFrame(train_features2, columns=train_dummy.columns)

scale large numeric values to the range of [0,1]

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
train_features_df[["HomePage","HomePage_Duration","LandingPage_Duration","ProductDescriptionPage_Duration"]] = scaler.fit_transform(train_features_df[[
    "HomePage", "HomePage_Duration","LandingPage_Duration","ProductDescriptionPage_Duration"]])

In [ ]:
imputer.fit(test_dummy)
test_1 = imputer.transform(test_dummy)
test_df = pd.DataFrame(test_1, columns=test_dummy.columns)
test_df[["HomePage","HomePage_Duration","LandingPage_Duration","ProductDescriptionPage_Duration"]] = scaler.fit_transform(test_df[[
    "HomePage", "HomePage_Duration","LandingPage_Duration","ProductDescriptionPage_Duration"]])

Use a classifier since we need to predict a label i.e. "Made_Purchase"
Compare the classifiers using evaluation metric (here cross_val_score)

In [ ]:
from sklearn.model_selection import cross_val_score

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
apply_abc=AdaBoostClassifier(n_estimators=200, learning_rate=0.01, random_state=20)
apply_abc.fit(train_features_df, train_label)

In [ ]:
cvs=cross_val_score(apply_abc, train_features_df, train_label, cv=30)
print("cross-validation score: ", cvs.mean())

In [ ]:
#from sklearn.tree import DecisionTreeClassifier
#apply_dtc = DecisionTreeClassifier()
#apply_dtc.fit(train_features_df, train_label)

In [ ]:
#cvs=cross_val_score(apply_dtc, train_features_df, train_label, cv=30)
#print("cross-validation score: ", cvs.mean())

In [ ]:
#from sklearn.ensemble import GradientBoostingClassifier
#apply_gbc = GradientBoostingClassifier()
#apply_gbc.fit(train_features_df, train_label)

In [ ]:
#cvs=cross_val_score(apply_gbc, train_features_df, train_label, cv=30)
#print("cross-validation score: ", cvs.mean())

In [ ]:
#from sklearn.neighbors import KNeighborsClassifier
#apply_knc = KNeighborsClassifier(n_neighbors = 3)
#apply_knc.fit(train_features_df, train_label)

In [ ]:
#cvs=cross_val_score(apply_knc, train_features_df, train_label, cv=30)
#print("cross-validation score: ", cvs.mean())

In [ ]:
#from sklearn.linear_model import SGDClassifier
#apply_sgd = SGDClassifier(eta0 = 0.01, learning_rate = 'constant')
#apply_sgd.fit(train_features_df, train_label)

In [ ]:
#cvs=cross_val_score(apply_sgd, train_features_df, train_label, cv=30)
#print("cross-validation score: ", cvs.mean())

In [ ]:
#from sklearn.ensemble import RandomForestClassifier
#apply_rfc = RandomForestClassifier()
#apply_rfc.fit(train_features_df, train_label)

In [ ]:
#cvs=cross_val_score(apply_rfc, train_features_df, train_label, cv=30)
#print("cross-validation score: ", cvs.mean())

In [ ]:
#from sklearn.ensemble import BaggingClassifier
#apply_bgc = BaggingClassifier(RandomForestClassifier(), max_samples = 0.5  , max_features = 0.5)
#apply_bgc.fit(train_features_df, train_label)

In [ ]:
#cvs=cross_val_score(apply_bgc, train_features_df, train_label, cv=30)
#print("cross-validation score: ", cvs.mean())

In [ ]:
predict_purchase=apply_abc.predict(test_df)
predict_purchase=predict_purchase.astype(bool)
predict_purchase

In [ ]:
submission=pd.DataFrame(columns=['id', 'Made_Purchase'])
submission['id']=[i for i in range(len(predict_purchase))]
submission["Made_Purchase"]=[i for i in predict_purchase]
submission.to_csv('submission.csv',index=False)
submission